In [2]:
from __future__ import print_function
import pandas as pd
import numpy as np
import re
from tqdm import tqdm,trange
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [3]:
os.chdir('C:\\Users\\ASUS\\PycharmProjects\\RahnemaProject')

In [4]:
df = pd.read_csv('feature_extracted_mid.csv')
df = df.drop('Unnamed: 0',axis=1)

In [ ]:
df.head()

In [5]:
pd.to_datetime(df['date_time'],unit='ms')

0         2021-05-12 00:36:00
1         2021-05-12 00:36:00
2         2021-05-12 00:36:00
3         2021-05-12 00:36:00
4         2021-05-12 00:36:00
                  ...        
1260030   2021-05-12 10:38:59
1260031   2021-05-12 10:38:59
1260032   2021-05-12 10:38:59
1260033   2021-05-12 10:38:59
1260034   2021-05-12 10:39:00
Name: date_time, Length: 1260035, dtype: datetime64[ns]

In [6]:
df = df.set_index(pd.to_datetime(df['date_time'],unit='ms'))
df = df.drop('date_time',axis=1)

In [7]:
#first_session = df.between_time('00:36:00','01:00:00').groupby(['client_ip',
 #   'user_agent']).mean()[['response_time','response_length','time_interval']]\
  #  .sort_values('response_time',ascending=False)

#for index,row in df[0:5].iterrows():
 #    i = pd.to_datetime(index).astype(int)/10**6

In [7]:
time_index = df.resample('3min')
#for name,group in time_index:
 #   print(name)

In [8]:
resampled_df = pd.DataFrame(time_index,columns=['date_time','columns'])
resampled_df = resampled_df.reset_index()
resampled_df = resampled_df.set_index('date_time')

In [9]:
resampled_df = resampled_df.drop('columns',axis=1)
resampled_df.head()

,index
date_time,
2021-05-12 00:36:00,0
2021-05-12 00:39:00,1
2021-05-12 00:42:00,2
2021-05-12 00:45:00,3
2021-05-12 00:48:00,4


In [12]:
resampled_df['index'].nunique()

202

In [10]:
resampled_df = pd.DataFrame(resampled_df.resample('s').ffill())

In [13]:
%time
#df = df.drop('session_number',axis=1)
#new_df = pd.concat([df,resampled_df],axis=1)
#new_df = df.reset_index()

new_df = pd.merge(df.reset_index(),resampled_df.reset_index(),left_on='date_time',right_on='date_time')

Wall time: 0 ns


In [ ]:
new_df = new_df.set_index('date_time')
#new_df = new_df.drop('date_time',axis=1)

new_df.head()

In [15]:
new_df.loc[new_df['geolocation'].isnull()==True,'geolocation'] = 'NaN'
new_df.loc[new_df['last_page'].isnull()==True,'last_page'] = 'NaN'

In [95]:
#new_df = new_df.reset_index()
new_df['date_time'] = pd.to_datetime(new_df['date_time'],'ms')
new_df['date_time'] = new_df['date_time'].apply(lambda x:x.value)
#new_df['date_time'] = pd.to_datetime(new_df['date_time']).astype(int)/10**6

In [96]:
new_df.info()


Define Features
---


In [26]:
session_1 = new_df.groupby(['index','client_ip','user_agent']).median()[['response_length','depth']]
session_2 = new_df.groupby(['index','client_ip','user_agent']).mean()[['response_time','time_interval']]

session_2.columns = ['avg_response_time','avg_time_interval_Std']

In [122]:
session_3 = new_df.groupby(['index','client_ip','user_agent']).std()[['response_time','time_interval']]
session_3.columns = ['response_time_std','time_interval_Std']

In [123]:
session_4 = new_df.groupby(['index','client_ip','user_agent']).sum()[['is_pc','Get','Post','image','400']]

In [124]:
session_5 = new_df.groupby(['index','client_ip','user_agent']).apply(lambda x:x.html.sum()/x.css_font_js.sum() if x.css_font_js.sum() !=0 else 0)
session_5 = pd.DataFrame(session_5,columns=['html/css'])

In [125]:
session_6 = new_df.groupby(['index','client_ip','user_agent']).apply(
    lambda x:(x['400'].sum()+x['404'].sum()+x['499'].sum())/x['200'].sum() if x['200'].sum()!=0 else 0)

session_6 = pd.DataFrame(session_6,columns=['4xx/200'])

In [24]:
session_7 = new_df.groupby(['index','client_ip','user_agent']).max()[['geolocation']]

In [127]:
session_8 = new_df.groupby(['index','client_ip','user_agent']).apply(
    lambda x:x.time.iloc[-1] - x.time.iloc[0])

session_8 = pd.DataFrame(session_8,columns=['lf_time_interval'])

In [129]:
session_9 = new_df.groupby(['index','client_ip','user_agent']).max()[['depth']]
session_9.columns = ['max_depth']
session_10 = new_df.groupby(['index','client_ip','user_agent']).std()[['depth']]
session_10.columns = ['std_depth']

In [138]:
# new_df['last_page'] = new_df.groupby(['client_ip','user_agent']).shift(1)['path']
# new_df.loc[new_df['last_page'].isnull()==True,'last_page']='NaN'
#
# new_df['return_count'] = new_df.apply(
#     lambda x:1  if
#         (len(x['last_page'].split('/'))>=2)
#         and
#         (x['path'].split('/')[-2] == x['last_page'].split('/')[-2])
#         else 0 , axis=1)
#
# new_df = new_df.drop('return_count',axis=1)

In [ ]:
session_11 = new_df.groupby(['index','client_ip','user_agent']).apply(
    lambda x: x.coded_path.split('-')[-1]
)

In [18]:
session_12 = new_df.groupby(['index','client_ip','user_agent']).apply(lambda x:x.is_pc.count())
#session_12 = new_df.groupby(['index','client_ip','user_agent']).count()['is_pc']
session_12 = pd.DataFrame(session_12, columns = ['number_of_requests'])

In [19]:
session_12['number_of_requests'].nunique()

304

In [119]:
#new_df = new_df.reset_index()
#new_df['time'] = new_df['date_time']
#new_df['date_time'] = pd.to_datetime(new_df['date_time'])
#new_df = new_df.set_index('date_time')
new_df.between_time('00:36','00:39').loc[(new_df.between_time('00:36','00:39')['client_ip']=='109.165.215.198')&(new_df.between_time('00:36','00:39')['user_agent']=='FreshpingBot/1.0 (+https://freshping.io/)')][['time']]

,time
date_time,
2021-05-12 00:36:43,1620779803000000000
2021-05-12 00:37:17,1620779837000000000
2021-05-12 00:38:56,1620779936000000000


In [ ]:
new_df.head()

Define Dataframe Based on Sessions
---

In [185]:
session_df = pd.concat([session_1,session_2,session_3,session_4,session_5,session_6,
                        session_7,session_8,session_9,session_10,session_12],axis=1)


In [187]:
pd.DataFrame(session_df).head()

response_length  \
index client_ip       user_agent                                                            
0     102.86.34.193   Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like ...              0.0   
                      Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Ma...              0.0   
      109.165.215.198 FreshpingBot/1.0 (+https://freshping.io/)                   63379.0   
      109.39.88.125   FreshpingBot/1.0 (+https://freshping.io/)                   63379.0   
      111.95.254.25   Mozilla/5.0 (Windows NT 10.0; Win64; x64) Apple...          22582.0   

                                                                          depth  \
index client_ip       user_agent                                                  
0     102.86.34.193   Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like ...    1.0   
                      Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Ma...    1.0   
      109.165.215.198 FreshpingBot/1.0 (+https://freshping.io/)             1.0   
      109.39.88.125   FreshpingBot/1.0 (+https://freshping.io/)             1.0   
      111.95.254.25   Mozilla/5.0 (Windows NT 10.0; Win64; x64) Apple...    1.0   

                                                                          avg_response_time  \
index client_ip       user_agent                                                              
0     102.86.34.193   Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like ...           4.571429   
                      Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Ma...           5.333333   
      109.165.215.198 FreshpingBot/1.0 (+https://freshping.io/)                   25.333333   
      109.39.88.125   FreshpingBot/1.0 (+https://freshping.io/)                   41.333333   
      111.95.254.25   Mozilla/5.0 (Windows NT 10.0; Win64; x64) Apple...           5.600000   

                                                                          avg_time_interval_Std  \
index client_ip       user_agent                                                                  
0     102.86.34.193   Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like ...           1.157700e+11   
                      Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Ma...           1.080520e+11   
      109.165.215.198 FreshpingBot/1.0 (+https://freshping.io/)                    5.402600e+11   
      109.39.88.125   FreshpingBot/1.0 (+https://freshping.io/)                    5.402600e+11   
      111.95.254.25   Mozilla/5.0 (Windows NT 10.0; Win64; x64) Apple...           3.241560e+11   

                                                                          response_time_std  \
index client_ip       user_agent                                                              
0     102.86.34.193   Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like ...           2.138090   
                      Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Ma...           4.450789   
      109.165.215.198 FreshpingBot/1.0 (+https://freshping.io/)                    9.237604   
      109.39.88.125   FreshpingBot/1.0 (+https://freshping.io/)                   11.547005   
      111.95.254.25   Mozilla/5.0 (Windows NT 10.0; Win64; x64) Apple...           3.577709   

                                                                          time_interval_Std  \
index client_ip       user_agent                                                              
0     102.86.34.193   Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like ...       4.331716e+11   
                      Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Ma...       4.184835e+11   
      109.165.215.198 FreshpingBot/1.0 (+https://freshping.io/)                9.357576e+11   
      109.39.88.125   FreshpingBot/1.0 (+https://freshping.io/)                9.357576e+11   
      111.95.254.25   Mozilla/5.0 (Windows NT 10.0; Win64; x64) Apple...       7.248348e+11   

                                                                          is_pc  \
index client_ip       user_agent                                 

In [188]:
session_df.to_csv('session_defined.csv')

In [26]:
x = pd.read_csv('session_defined.csv')
x = x.set_index(['index','client_ip','user_agent'])
#x = pd.concat([x,session_7],axis=1)

In [ ]:
x.head()


In [27]:
x.to_csv('session_defined.csv')

